In [27]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp
import time
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)


In [6]:
(repo_id, model_file_name) = ("",
                                  "ggml-model27-q4.gguf")

model_path = hf_hub_download(repo_id=repo_id,
                                 filename=model_file_name,
                                 repo_type="model")

llm = LlamaCpp(
        model_path=model_path,
        n_gpu_layers=512,
        n_batch=30,
        n_ctx=6000,
        max_tokens=200,
        temperature=0,
        # callback_manager=callback_manager,
        verbose=True,
        streaming=True,
        stop=["Human:","HUMAN", "###","You are a personal assistant","Hi!"]
        )

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
Model metadata: {'general.name': '7B', 'general.architecture': 'llama', 'llama.context_length': '32768', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '4096', 'llama.block_count': '32', 'llama.feed_forward_length': '14336', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '3', 'llama.attention.head_count_kv': '8', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.freq_base': '10000.000000', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.padding_token_id': '2', 'tokenizer.ggml.add_bos_token': 'true', 'tokenizer.ggml.add_eos_token': 'false'}


In [28]:
loader = PyPDFLoader("_RAG.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)
hf_embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
db = FAISS.from_documents(docs, hf_embedding)
db.save_local("_FAISS")
db = FAISS.load_local("_FAISS/", embeddings=hf_embedding)

retriever = db.as_retriever(search_type = 'similarity', search_kwargs={"k":2})

CUDA SETUP: Loading binary c:\Users\berkg\anaconda3\envs\torch\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...
Could not find module 'c:\Users\berkg\anaconda3\envs\torch\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll' (or one of its dependencies). Try using the full path with constructor syntax.
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CUDA SETUP: Solution 2): If you do not have sudo rights, you can do the following:
CUDA SETUP: Solution 2a): Find the cuda library via: find / -name libcuda.so 2>/dev/null
CUDA SETUP: Solution 2b): Once the library is found add it to the LD_LIBRARY_PATH: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:FOUND_PATH_FROM_2a
CUDA SETUP: Solution 2c): For a permanent solution add the export from 2b into your .bashrc file, located at ~/.bashrc
CUDA SETUP: Problem: The

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Inspect the CUDA SETUP outputs above to fix your environment!
        If you cannot find any issues and suspect a bug, please open an issue with detals about your environment:
        https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
template = (
    """Combine the chat history and follow up question into
    a standalone question. Chat History: {chat_history}"
    Follow up question: {question}"""
)
prompt = PromptTemplate.from_template(template)


In [ ]:
memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

In [ ]:
question_generator_chain = LLMChain(llm=llm, prompt=prompt)

chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=False)
conv_qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory)